In [10]:
import io
import pandas as pd
import torch
import torch.nn as nn
from transformers import AutoModel, AutoTokenizer
from pyvi import ViTokenizer
from dotenv import load_dotenv
import os
import warnings
class EmotionClassifier(nn.Module):
    def __init__(self, model_name, n_classes=7):
        super(EmotionClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained(model_name)
        self.drop = nn.Dropout(p=0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        _, output = self.bert(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        x = self.drop(output)
        return self.fc(x)

def initialize_model(model_name, model_path, n_classes=7):
    model = EmotionClassifier(model_name, n_classes).to(device)
    state_dict = torch.load(model_path, map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    return model
def load_model_from_path(model_path):
    with open(model_path, 'rb') as f:
        buffer = io.BytesIO(f.read())
    return buffer

def initialize_model(model_name, model_path, n_classes=7):
    model = EmotionClassifier(model_name, n_classes).to(device)
    model_buffer = load_model_from_path(model_path)
    state_dict = torch.load(model_buffer, map_location=device)
    model.load_state_dict(state_dict, strict=False)
    model.eval()
    return model

class_names = ['Enjoyment', 'Disgust', 'Sadness', 'Anger', 'Surprise', 'Fear', 'Other']
def infer(text, tokenizer, model, max_len=120):
    text = ViTokenizer.tokenize(text)
    encoded = tokenizer.encode_plus(
        text, max_length=max_len, truncation=True, add_special_tokens=True,
        padding='max_length', return_attention_mask=True, return_tensors='pt'
    )
    input_ids = encoded['input_ids'].to(device)
    attention_mask = encoded['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    _, y_pred = torch.max(output, dim=1)
    return output,class_names[y_pred]

# Initialize device and class names
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
tokenizer = AutoTokenizer.from_pretrained("uitnlp/visobert")
model = initialize_model("uitnlp/visobert", 'models/visobert_fold6.pth')

Some weights of XLMRobertaModel were not initialized from the model checkpoint at uitnlp/visobert and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\APTS\AppData\Local\Temp\ipykernel_19080\3719082247.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.seri

In [33]:
infer('Điều đáng sợ nhất đối với mỗi người là mỗi ngày thức dậy. Và nhận ra trong cuộc sống mình không có người và điều gì để chờ đợi, cố gắng.', tokenizer,model)

(tensor([[-1.8984, -2.6308,  3.0371, -0.7220, -1.2088,  3.5465,  0.5451]],
        grad_fn=<AddmmBackward0>),
 'Fear')

In [21]:
def test(model,test_path):
    #test and print record that not true
    df = pd.read_csv(test_path)
    df['predict']=
    if(df['predict']!=df['emotion']):
        print(df[df['predict']!=df['label']])
    return df

(tensor([[-1.1149, -1.6407, -0.6472, -1.2589,  3.4264, -0.8914,  2.0067]],
        grad_fn=<AddmmBackward0>),
 'Surprise')